In [8]:
!pip install pytorch_tabnet
!pip install hyperopt

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

train_df = pd.read_csv('/content/drive/MyDrive/CTGAN_oversampled_train_1_1.csv',encoding='utf-8')  ###############################
test_df = pd.read_csv('/content/drive/MyDrive/최종_동태_test.csv',encoding='utf-8')


# Assume that the last column is the target and the rest are features
X = train_df.iloc[:,:-1]
y = train_df.iloc[:, -1]

scaler.fit(X)
X = scaler.transform(X)

# Split the data into training and validation sets (80:20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to numpy arrays for TabNet
X_train = X_train
y_train = y_train
X_val = X_val
y_val = y_val


In [22]:
train_df.iloc[:,:-1]

,총자산경상이익률,총자산순이익률,금융비용/매출액,금융비용/총부채,순금융비용/매출액,매출액경상이익률,매출액순이익률,자기자본경상이익률,자기자본순이익률,EBITDA/이자비용,...,이익잉여금/총자산_변화량,이익잉여금/총부채_변화량,이익잉여금/유동자산_변화량,현금비율_변화량,당좌비율_변화량,유동비율_변화량,유동부채회전율_변화량,총자산_변화량,매출액_변화량,고정자산_변화량
0,1.400000,0.520000,0.016500,0.020759,0.016518,3.880000,1.450000,1.960000,0.740000,6.977523e+05,...,0.317284,-0.207783,-0.254297,325.820000,-39.240000,1.220000,1.316550,1.430400e+10,4.147000e+09,7.704000e+09
1,0.780000,0.280000,-0.006500,-0.018509,-0.006461,1.940000,0.690000,0.900000,0.320000,3.596125e+06,...,0.004411,-3.337870,-0.485126,-293.390000,-342.510000,-370.130000,-1.099866,4.800000e+08,2.351000e+09,1.206400e+10
2,-4.490000,-6.370000,-0.001100,-0.003166,-0.001109,-9.510000,-13.500000,-5.370000,-7.620000,-3.215209e+07,...,-0.065183,0.552992,-0.295992,-72.240000,-33.020000,-27.730000,0.446005,3.100000e+08,4.289000e+09,4.940000e+08
3,2.070000,2.060000,-0.004700,-0.015814,-0.004671,5.250000,5.240000,2.340000,2.340000,1.839779e+11,...,0.333272,0.437456,-7428.352695,-5.480000,556.960000,565.300000,0.552812,2.645600e+10,5.928000e+09,1.947000e+09
4,1.410000,2.570000,-0.027200,-0.105115,-0.027181,3.750000,6.830000,1.570000,2.850000,1.839779e+11,...,0.452513,5.250437,0.000000,11.380000,-118.510000,-115.870000,0.207839,2.651750e+10,8.566000e+09,-3.867000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262611,4.220464,28.661857,-0.022087,0.021840,-0.003883,44.077079,4.376668,-150.497079,3.379074,1.603160e+09,...,-0.225593,-0.115148,-22.848917,47.891736,3624.997826,-51.082228,-2.174777,1.929359e+09,-2.493049e+10,-1.334017e+08
262612,-3.491165,3.824467,0.099023,0.048209,0.041389,-6.667071,-18.730185,31.637270,-9.688987,-2.038276e+09,...,-0.063538,-0.198053,-12.457046,-10.459174,-47.964826,-80.908113,-0.181034,2.747192e+09,-2.794841e+09,-6.479318e+08
262613,-0.731232,4.189051,-0.017512,0.019466,-0.056311,18.898232,-21.518412,-19.685213,32.644088,7.960788e+08,...,-0.157116,-0.035649,-61.603011,-58.826530,28.281616,-51.954252,0.322735,6.456113e+09,-5.246069e+09,-3.904229e+09
262614,14.629055,12.287365,-0.015072,0.045999,-0.028462,35.946646,-356.297678,-8.099356,28.439918,-9.428074e+08,...,0.062206,0.077493,-21.808188,32.759173,24.250765,-200.364791,-1.966277,3.374012e+09,-4.886925e+09,-8.027666e+08


In [23]:
import torch

# GPU 사용 가능한지 확인하고 설정합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [24]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score,f1_score
import numpy as np
from sklearn.metrics import precision_recall_curve

# Define the search space
space = {
    'n_d': hp.choice('n_d', [8, 16, 24, 32]),
    'n_a': hp.choice('n_a', [8, 16, 24, 32]),
}

def objective(params):
    # Ensure parameters that should be integers are integers
    for parameter_name in ['n_d', 'n_a']:
        params[parameter_name] = int(params[parameter_name])

    clf = TabNetClassifier(
        n_d=params['n_d'],
        n_a=params['n_a'],
        device_name='cuda'
    )

    clf.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        patience=30,
        max_epochs=200,
    )

    preds_proba = clf.predict_proba(X_val)[:, 1]

    # Compute Precision-Recall curve and find optimal threshold
    precision, recall, thresholds = precision_recall_curve(y_val, preds_proba)
    f1_scores = 2*(precision*recall)/(precision+recall)
    f1_scores = f1_scores[~np.isnan(f1_scores)]  # Remove any NaN F1 scores

    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]

    preds = (preds_proba > optimal_threshold).astype(int)
    f1 = f1_score(y_val, preds)

    return {'loss': -f1, 'status': STATUS_OK}

# Run the algorithm
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=15,
            trials=trials)

print("Best: ", best)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")



epoch 0  | loss: 0.34826 | val_0_auc: 0.96682 |  0:00:08s
epoch 1  | loss: 0.14676 | val_0_auc: 0.98655 |  0:00:17s
epoch 2  | loss: 0.1196  | val_0_auc: 0.98878 |  0:00:26s
epoch 3  | loss: 0.10604 | val_0_auc: 0.99074 |  0:00:35s
epoch 4  | loss: 0.09839 | val_0_auc: 0.99158 |  0:00:44s
epoch 5  | loss: 0.09334 | val_0_auc: 0.99143 |  0:00:53s
epoch 6  | loss: 0.09226 | val_0_auc: 0.99182 |  0:01:02s
epoch 7  | loss: 0.08653 | val_0_auc: 0.99204 |  0:01:11s
epoch 8  | loss: 0.08696 | val_0_auc: 0.99234 |  0:01:20s
epoch 9  | loss: 0.08341 | val_0_auc: 0.99279 |  0:01:29s
epoch 10 | loss: 0.08153 | val_0_auc: 0.99314 |  0:01:38s
epoch 11 | loss: 0.08663 | val_0_auc: 0.99291 |  0:01:47s
epoch 12 | loss: 0.08105 | val_0_auc: 0.99305 |  0:01:56s
epoch 13 | loss: 0.08028 | val_0_auc: 0.99267 |  0:02:05s
epoch 14 | loss: 0.08302 | val_0_auc: 0.99298 |  0:02:13s
epoch 15 | loss: 0.07852 | val_0_auc: 0.9936  |  0:02:23s
epoch 16 | loss: 0.08229 | val_0_auc: 0.99358 |  0:02:31s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



  7%|▋         | 1/15 [13:47<3:13:11, 827.95s/trial, best loss: -0.9782119872838646]

...

 93%|█████████▎| 14/15 [3:41:08<15:21, 921.51s/trial, best loss: -0.9785816646655564]

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")



epoch 0  | loss: 0.28513 | val_0_auc: 0.9488  |  0:00:08s
epoch 1  | loss: 0.12584 | val_0_auc: 0.99034 |  0:00:17s
epoch 2  | loss: 0.11087 | val_0_auc: 0.99102 |  0:00:26s
epoch 3  | loss: 0.10149 | val_0_auc: 0.99123 |  0:00:35s
epoch 4  | loss: 0.09161 | val_0_auc: 0.99235 |  0:00:44s
epoch 5  | loss: 0.08726 | val_0_auc: 0.99299 |  0:00:53s
epoch 6  | loss: 0.08476 | val_0_auc: 0.99333 |  0:01:02s
epoch 7  | loss: 0.08353 | val_0_auc: 0.99363 |  0:01:11s
epoch 8  | loss: 0.08027 | val_0_auc: 0.99366 |  0:01:20s
epoch 9  | loss: 0.07887 | val_0_auc: 0.99372 |  0:01:29s
epoch 10 | loss: 0.07806 | val_0_auc: 0.9938  |  0:01:38s
epoch 11 | loss: 0.07708 | val_0_auc: 0.99397 |  0:01:47s
epoch 12 | loss: 0.07748 | val_0_auc: 0.9936  |  0:01:57s
epoch 13 | loss: 0.07538 | val_0_auc: 0.99407 |  0:02:05s
epoch 14 | loss: 0.07628 | val_0_auc: 0.99359 |  0:02:14s
epoch 15 | loss: 0.0759  | val_0_auc: 0.99397 |  0:02:23s
epoch 16 | loss: 0.07548 | val_0_auc: 0.99411 |  0:02:32s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



100%|██████████| 15/15 [3:51:51<00:00, 927.43s/trial, best loss: -0.9785816646655564]
Best:  {'n_a': 0, 'n_d': 3}


In [25]:
print(best)

{'n_a': 0, 'n_d': 3}


In [26]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# Convert indices to parameters
best_params = {
    'n_d': [8, 16, 24, 32][best['n_d']],
    'n_a': [8, 16, 24, 32][best['n_a']],
}

# Train model with best parameters
clf = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
)

clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    patience=30,
    max_epochs=200,
)

# Evaluate model
preds = clf.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, preds)
print("Best model ROC AUC: ", auc)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.31276 | val_0_auc: 0.95096 |  0:00:09s
epoch 1  | loss: 0.13008 | val_0_auc: 0.98882 |  0:00:17s
epoch 2  | loss: 0.10735 | val_0_auc: 0.99048 |  0:00:26s
epoch 3  | loss: 0.09925 | val_0_auc: 0.9917  |  0:00:35s
epoch 4  | loss: 0.09521 | val_0_auc: 0.99171 |  0:00:44s
epoch 5  | loss: 0.09468 | val_0_auc: 0.99188 |  0:00:53s
epoch 6  | loss: 0.0884  | val_0_auc: 0.99249 |  0:01:02s
epoch 7  | loss: 0.08937 | val_0_auc: 0.9924  |  0:01:11s
epoch 8  | loss: 0.08548 | val_0_auc: 0.99286 |  0:01:20s
epoch 9  | loss: 0.08269 | val_0_auc: 0.99294 |  0:01:29s
epoch 10 | loss: 0.08141 | val_0_auc: 0.99316 |  0:01:38s
epoch 11 | loss: 0.07938 | val_0_auc: 0.9933  |  0:01:47s
epoch 12 | loss: 0.084   | val_0_auc: 0.99309 |  0:01:56s
epoch 13 | loss: 0.08559 | val_0_auc: 0.99201 |  0:02:05s
epoch 14 | loss: 0.08966 | val_0_auc: 0.99236 |  0:02:14s
epoch 15 | loss: 0.08235 | val_0_auc: 0.99333 |  0:02:23s
epoch 16 | loss: 0.07936 | val_0_auc: 0.99339 |  0:02:32s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Best model ROC AUC:  0.9949315651900039


In [27]:
test_df.iloc[:, 3:-1]

,총자산경상이익률,총자산순이익률,금융비용/매출액,금융비용/총부채,순금융비용/매출액,매출액경상이익률,매출액순이익률,자기자본경상이익률,자기자본순이익률,EBITDA/이자비용,...,이익잉여금/총자산_변화량,이익잉여금/총부채_변화량,이익잉여금/유동자산_변화량,현금비율_변화량,당좌비율_변화량,유동비율_변화량,유동부채회전율_변화량,총자산_변화량,매출액_변화량,고정자산_변화량
0,0.84,1.60,-0.0254,-0.122986,-0.025390,2.39,4.56,0.91,1.73,7.907038e+07,...,0.015472,0.184095,0.031436,-24.28,-141.42,-155.55,0.337766,3.466000e+09,5.015000e+09,2.431900e+10
1,1.45,-0.19,-0.0237,-0.091076,-0.023699,3.63,-0.47,1.61,-0.21,1.987598e+08,...,-0.000921,-0.191910,-0.003426,-14.32,-298.31,-297.30,-1.307682,8.952000e+09,1.272200e+10,4.497000e+09
2,29.25,23.23,0.0006,0.001761,0.000592,32.86,26.09,42.00,33.35,1.216048e+08,...,0.146424,0.741189,-0.123487,36.13,67.74,67.62,0.046018,2.305240e+11,2.787620e+11,1.540300e+10
3,41.86,30.43,-0.0068,-0.024544,-0.006799,45.89,33.36,56.02,40.73,2.322498e+08,...,0.133972,0.932836,0.008106,-17.16,86.71,85.27,0.520850,4.720595e+11,4.606060e+11,6.020000e+10
4,0.03,0.60,0.0044,0.000687,0.006050,0.23,4.27,0.31,11.62,3.557666e+05,...,0.020923,0.024074,0.012722,488.70,757.69,811.07,5.032471,3.093865e+11,4.511500e+10,-3.254960e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38238,13.79,11.92,-0.0053,-0.017173,-0.005275,11.40,9.86,22.00,19.02,1.839779e+11,...,-0.111046,-0.676866,-7431.507309,-58.36,-42.92,-42.92,-0.883486,5.942000e+09,6.175000e+09,9.220000e+08
38239,-1.90,-3.72,0.0716,0.026644,0.071566,-7.57,-14.79,-5.89,-11.50,1.190084e+00,...,-0.063896,0.018403,12.937910,12.01,11.49,11.33,0.334871,-1.029500e+09,1.652000e+09,-1.153000e+09
38240,5.27,3.85,0.0466,0.022394,0.046637,16.38,11.96,15.96,11.65,5.976569e+00,...,0.027306,0.026300,2.025173,2.73,3.97,4.01,0.354836,-3.910000e+08,2.045000e+09,-5.620000e+08
38241,5.04,4.29,-0.0010,-0.003636,-0.001040,3.09,2.63,9.13,7.77,3.521770e+01,...,0.038513,0.048395,0.065115,-31.84,-21.77,-22.69,-0.319614,5.591000e+09,5.698000e+09,3.170000e+08


In [28]:
from sklearn.metrics import roc_auc_score
from hyperopt import space_eval

X_test = test_df.iloc[:, 3:-1]
y_test = test_df.iloc[:, -1]
X_test, y_test = X_test.values, y_test.values

X_test_scaled = scaler.transform(X_test)

# 테스트 데이터에 대한 예측 생성
preds_test = clf.predict_proba(X_test_scaled)[:, 1]

# ROC AUC 점수 계산
roc_auc = roc_auc_score(y_test, preds_test)
print("ROC AUC score on test data: ", roc_auc)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ROC AUC score on test data:  0.8561638770731713


In [ ]:
# import torch

# torch.save(clf, '최종_이상치처리후_best_model.pth')

# # Load the entire model
# clf = torch.load('best_tabnet_model.pth')


In [29]:
# from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve, auc

# thresholds = np.arange(0, 1.1, 0.1)  # 0부터 1까지 0.1 단위로 임계값 설정

# for threshold in thresholds:
#     # 예측 확률을 기준으로 예측 레이블 생성
#     preds_label = (preds_test >= threshold).astype(int)

#     print(f"Threshold: {threshold}")
#     print("Confusion Matrix:")
#     print(confusion_matrix(y_test, preds_label))
#     print("Accuracy Score: ", accuracy_score(y_test, preds_label))
#     print("Precision Score: ", precision_score(y_test, preds_label))
#     print("Recall Score: ", recall_score(y_test, preds_label))
#     print("ROC AUC Score: ", roc_auc_score(y_test, preds_test))  # 주의: ROC AUC는 예측 확률을 사용
#     print("F1 Score: ", f1_score(y_test, preds_label))
#     print("-" * 50)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve, auc, fbeta_score

precision, recall, thresholds = precision_recall_curve(y_test, preds_test)

beta=2
f2_scores = ((1 + beta**2) * (precision * recall)) / ((beta**2 * precision) + recall)
f2_scores = f2_scores[~np.isnan(f2_scores)]
optimal_idx = np.argmax(f2_scores)
optimal_threshold = thresholds[optimal_idx]

# thresholds = np.clip(np.linspace(optimal_threshold - 0.1, optimal_threshold + 0.1, 40), 0, 1)
# for threshold in thresholds:

# predictions = (probas[:, 1] >= optimal_threshold).astype('int')



# f1_scores = 2*(precision*recall)/(precision+recall)
# f1_scores = f1_scores[~np.isnan(f1_scores)]

optimal_idx = np.argmax(f2_scores)
optimal_threshold = thresholds[optimal_idx]



# thresholds = np.clip(np.linspace(optimal_threshold - 0.1, optimal_threshold + 0.1, 40), 0, 1)
# for threshold in thresholds:

predictions = (preds_test >= optimal_threshold).astype('int')

# 테스트 데이터에 대한 예측 성능 지표를 계산
test_cm = confusion_matrix(y_test, predictions)
test_acc = accuracy_score(y_test, predictions)
test_pre = precision_score(y_test, predictions)
test_rcll = recall_score(y_test, predictions)
test_roc_auc = roc_auc_score(y_test, preds_test)
test_f1 = f1_score(y_test, predictions)
test_f2 = fbeta_score(y_test, predictions, beta=2)

# 테스트 데이터에 대한 예측 성능 출력
print('Threshold :', round(optimal_threshold, 2))
print('Confusion Matrix :', test_cm)
print('Accuracy Score :', round(test_acc,4))
print('Precision Score :', round(test_pre,4))
print('Recall Score :', round(test_rcll,4))
print('ROC AUC Score :', round(test_roc_auc,4))
print('F1 Score :', round(test_f1,4))
print('f2 스코어 :', round(test_f2,4))
print('\n')

Threshold : 0.04
Confusion Matrix : [[30203  6252]
 [  475  1313]]
Accuracy Score : 0.8241
Precision Score : 0.1736
Recall Score : 0.7343
ROC AUC Score : 0.8562
F1 Score : 0.2808
f2 스코어 : 0.4461




<ipython-input-29-32472b63feb7>:25: RuntimeWarning: invalid value encountered in true_divide
  f2_scores = ((1 + beta**2) * (precision * recall)) / ((beta**2 * precision) + recall)
